In [25]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.

cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='yvdwS_cuX1y9Aa6x9p-491afybRMNN3NPJ7pDOpKv9mZ',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/identity/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.direct.au-syd.cloud-object-storage.appdomain.cloud')

bucket = 'watersource-donotdelete-pr-ckewxzwej8suuj'
object_key = 'nss Items data.csv'

body = cos_client.get_object(Bucket=bucket,Key=object_key)['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_1 = pd.read_csv(body)
df_1.head(10)


,State,Age Group,Sector,Gender,Indicator,Value
0,All India,15 years and above,All,Male,Percentage of Persons Who Used Mobile Telephon...,83.2
1,All India,15 years and above,All,Female,Percentage of Persons Who Used Mobile Telephon...,56.7
2,All India,15 years and above,All,Person,Percentage of Persons Who Used Mobile Telephon...,70.2
3,All India,15 years and above,Rural,Male,Percentage of Persons Who Used Mobile Telephon...,80.2
4,All India,15 years and above,Rural,Female,Percentage of Persons Who Used Mobile Telephon...,49.9
5,All India,15 years and above,Rural,Person,Percentage of Persons Who Used Mobile Telephon...,65.3
6,All India,15 years and above,Urban,Male,Percentage of Persons Who Used Mobile Telephon...,90.0
7,All India,15 years and above,Urban,Female,Percentage of Persons Who Used Mobile Telephon...,72.4
8,All India,15 years and above,Urban,Person,Percentage of Persons Who Used Mobile Telephon...,81.4
9,All India,18 years and above,All,Male,Percentage of Persons Who Used Mobile Telephon...,86.6


In [6]:
# Check dataset info
df_1.info()

# Check column names
print(df_1.columns)

# Descriptive statistics
df_1.describe()

# Missing values
df_1.isnull().sum()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 666 entries, 0 to 665
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   State      666 non-null    object 
 1   Age Group  666 non-null    object 
 2   Sector     666 non-null    object 
 3   Gender     666 non-null    object 
 4   Indicator  666 non-null    object 
 5   Value      666 non-null    float64
dtypes: float64(1), object(5)
memory usage: 31.3+ KB
Index(['State', 'Age Group', 'Sector', 'Gender', 'Indicator', 'Value'], dtype='object')


State        0
Age Group    0
Sector       0
Gender       0
Indicator    0
Value        0
dtype: int64

In [7]:
# Filter only records about drinking water
df_water = df_1[df_1['Indicator'].str.contains("Drinking Water", case=False, na=False)]
df_water.head()


,State,Age Group,Sector,Gender,Indicator,Value


In [9]:
# Display first few records
df_1.head()

# Check dataset size
print("Rows:", df_1.shape[0], "Columns:", df_1.shape[1])

# View column names
print(df_1.columns)

# Get unique indicator types
print(df_1['Indicator'].unique())


Rows: 666 Columns: 6
Index(['State', 'Age Group', 'Sector', 'Gender', 'Indicator', 'Value'], dtype='object')
['Percentage of Persons Who Used Mobile Telephone with Active Sim Card at Least Once During Last Three Months Preceding the Date of the Survey']


In [10]:
# Filter for drinking water related rows
df_water = df_1[df_1['Indicator'].str.contains("Drinking Water", case=False, na=False)]
df_water.head()


,State,Age Group,Sector,Gender,Indicator,Value


In [14]:
import seaborn as sns
import matplotlib.pyplot as plt

# Clean column names
df_water.columns = df_water.columns.str.strip()

# Filter by 'water' broadly
df_water = df_1[df_1['Indicator'].str.contains("water", case=False, na=False)]

# Check if data exists
if df_water.empty:
    print("No matching data found for drinking water.")
else:
    state_water = df_water.groupby('State')['Value'].mean().sort_values(ascending=False)
    
    plt.figure(figsize=(10,6))
    state_water.plot(kind='barh', color='skyblue')
    plt.xlabel('Average % with Access to Improved Drinking Water')
    plt.title('State-wise Access to Improved Drinking Water')
    plt.show()


No matching data found for drinking water.


In [16]:
# ------------------------------------------------------
# 📊 STEP 4: Safe check before plotting
# ------------------------------------------------------

if df_water.empty:
    print("❗ No data found for drinking water indicators. Please check your filter.")
else:
    print(f"✔️ Drinking Water records found: {df_water.shape[0]}")
    display(df_water.head())


❗ No data found for drinking water indicators. Please check your filter.


In [18]:
# ------------------------------------------------------
# 📥 STEP 1: Import Libraries
# ------------------------------------------------------
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Optional: Style
sns.set_theme(style="whitegrid")
plt.rcParams['axes.titlesize'] = 14
plt.rcParams['axes.labelsize'] = 12

# ------------------------------------------------------
# 📥 STEP 2: Load Data (already done by your earlier cell)
# df_1 = your main dataset
# ------------------------------------------------------

# Clean column names: remove any extra spaces
df_1.columns = df_1.columns.str.strip()

# ------------------------------------------------------
# 📊 STEP 3: Filter for Drinking Water Data
# ------------------------------------------------------
# Try to match any water-related indicator
df_water = df_1[df_1['Indicator'].str.contains("water", case=False, na=False)]

# Check data
print(f"Total Drinking Water records found: {df_water.shape[0]}")
display(df_water.head())

# ------------------------------------------------------
# ✅ STEP 4: Safe check before plotting
# ------------------------------------------------------
if df_water.empty:
    print("❗ No data found for drinking water indicators. Check your dataset.")
else:
    # ------------------------------------------------------
    # 📈 State-wise Analysis
    # ------------------------------------------------------
    plt.figure(figsize=(12, 8))
    state_water = df_water.groupby('State')['Value'].mean().sort_values(ascending=False)
    state_water.plot(kind='barh', color='skyblue')
    plt.xlabel('Average % Population with Access')
    plt.title('State-wise Access to Improved Drinking Water')
    plt.show()

    # ------------------------------------------------------
    # 📈 Sector-wise (Rural/Urban)
    # ------------------------------------------------------
    plt.figure(figsize=(6,4))
    sector_water = df_water.groupby('Sector')['Value'].mean()
    sector_water.plot(kind='bar', color=['lightgreen', 'salmon'])
    plt.ylabel('% Population')
    plt.title('Rural vs Urban Access to Improved Drinking Water')
    plt.show()

    # ------------------------------------------------------
    # 📈 Gender-wise
    # ------------------------------------------------------
    plt.figure(figsize=(6,4))
    gender_water = df_water.groupby('Gender')['Value'].mean()
    gender_water.plot(kind='bar', color=['orange','purple'])
    plt.ylabel('% Population')
    plt.title('Access to Improved Drinking Water by Gender')
    plt.show()

    # ------------------------------------------------------
    # 📋 List States with Lowest Access
    # ------------------------------------------------------
    print("📉 States with lowest average access to improved drinking water:")
    print(state_water.tail(5))

# ------------------------------------------------------
# ✅ END OF PROJECT ANALYSIS
# ------------------------------------------------------
print("✔️ Drinking Water Analysis completed.")


Total Drinking Water records found: 0


,State,Age Group,Sector,Gender,Indicator,Value


❗ No data found for drinking water indicators. Check your dataset.
✔️ Drinking Water Analysis completed.


### 📊 Insights:
- Top 5 states: X, Y, Z, A, B
- Rural population access is lower by __%
- Urban access is generally higher by __%
- Gender gap is negligible / significant
- Focus needed in states with <50% access
